# MATH50003 (2023–24)
# Lab 7: III.5 Orthogonal and Unitary Matrices and III.6 QR Factorisation

This lab explores orthogonal matrices, including rotations and reflections.
We will construct special types to capture the structure of these orthogonal operations,
with the goal of implementing fast matrix*vector and matrix\vector operations.
We also compute the QR factorisation with Householder reflections, and use this
to solve least squares problems.

**Learning Outcomes**

Mathematical knowledge:

1. Constructing rotation and reflection matrices.
2. Computing the QR factorisation using reflections.
3. Computing a tridiagonal QR factorisation using rotations.
4. The relationship between QR and least squares.

Coding knowledge:

1. The `atan(y,x)` function and the `I` convenience syntax.
2. Templating fields in a type.
3. Using the `qr` function to solve least squares.

In [2]:
using LinearAlgebra, Test

## III.5 Orthogonal and Unitary Matrices

Here we explore representing rotations and reflections, which are
special types of orthogonal/unitary matrices.

### III.5.1 Rotations

A (simple) rotation matrix is an element of the special orthogonal group $SO(2)$ and has a matrix representation
$$
 \begin{bmatrix} c & -s \\ s & c \end{bmatrix}
$$
such that $c^2 + s^2 = 1$.
More generally, we can generalise simple rotations on higher dimensional vectors by acting on two indices at a time.
There are multiple ways of storing a rotation matrix, here we explore the most intuitive (but not the fastest!) way of storing just an angle $θ$
so that $c = \cos θ$ and $s = \sin θ$.

We will use a syntax in a struct that forces a field to be a special type. In what follows we define
the `getindex` by first implementing multiplication, a pattern that will be reused in the problems.

In [3]:
struct Rotation <: AbstractMatrix{Float64}
    θ::Float64 # The ::Float64 means θ can only be a Float64
end

import Base: *, size, getindex

size(Q::Rotation) = (2, 2)

function *(Q::Rotation, x::AbstractVector)
    if length(x) ≠ 2
        error("dimension mismatch")
    end
    θ = Q.θ
    c,s = cos(θ), sin(θ)
    a,b = x # special syntax so that a == x[1] and b == x[2]
    [c*a - s*b, s*a + c*b]
end

function getindex(Q::Rotation, k::Int, j::Int)
    # We use the overloaded * above as we will follow this pattern later.
    e_k = zeros(2)
    e_j = zeros(2)
    e_k[k] = 1  # will error if k ≠ 1 or 2
    e_j[j] = 1  # will error if j ≠ 1 or 2
    e_k'*(Q*e_j)
end

Q = Rotation(0.1)

2×2 Rotation:
 0.995004   -0.0998334
 0.0998334   0.995004

We can test the ability to rotate a vector to the $x$-axis. Here we use the inbuilt `atan(y,x)` function
to compute the angle of a vector:

In [4]:
x = [-1,-2]
θ = atan(x[2], x[1]) # angle of the vector x
Q = Rotation(-θ) # rotate back
Q * x # first entry is norm(x), second entry is 0

2-element Vector{Float64}:
 2.23606797749979
 0.0

-----

**Problem 1** Complete the implementation of `Rotations`, which represents an orthogonal matrix `Q` that is a product
of rotations of angle `θ[k]`, each acting on the entries `k:k+1`. That is, it returns $Q = Q_1⋯Q_k$ such that
$$
Q_k[k:k+1,k:k+1] =
\begin{bmatrix}
\cos θ[k] & -\sin θ[k]\\
\sin θ[k] & \cos θ[k]
\end{bmatrix}
$$
with all other entries being equivalent to the identity.

In [5]:
struct Rotations <: AbstractMatrix{Float64}
    θ::Vector{Float64} # a vector of angles
end

# a1 -b1  0   0
# b1  a1  0   0
#  0   0 a2 -b2
#  0   0 b2  a2

# we use the number of rotations to deduce the dimensions of the matrix
size(Q::Rotations) = (length(Q.θ)+1, length(Q.θ)+1)

function *(Q::Rotations, x::AbstractVector)
    # TODO: Apply Q in O(n) operations. You may assume x has Float64 entries.
    # Hint: you may wish to use copy(x) and only change the relevant entries.
    y = copy(x) # copies x to a new Vector
    θ = Q.θ

    for k = length(θ):-1:1
        c,s = cos(θ[k]), sin(θ[k])
        y[k], y[k+1] = c*y[k] - s*y[k+1], s*y[k] + c*y[k+1]
    end


    y
end

function getindex(Q::Rotations, k::Int, j::Int)
    n, _ = size(Q)
    x = zeros(n)
    x[j] = 1
    y = Q*x
    y[k]
end

θ = randn(5)
Q = Rotations(θ)
@test Q'Q ≈ I
@test Rotations([π/2, -π/2]) ≈ [0 0 -1; 1 0 0; 0 -1 0]

# 0  0 -1
# 1  0  0
# 0 -1  0

Test Passed
  Expression: Rotations([π / 2, -π / 2]) ≈ [0 0 -1; 1 0 0; 0 -1 0]
   Evaluated: [6.123233995736766e-17 -6.123233995736766e-17 -1.0; 1.0 3.749399456654644e-33 6.123233995736766e-17; 0.0 -1.0 6.123233995736766e-17] ≈ [0 0 -1; 1 0 0; 0 -1 0]

------

### III.5.2 Reflections

We can also construct reflections, defined by a normalised vector $𝐯$ as
$$
Q_{𝐯} := I - 2𝐯𝐯^⋆
$$
The obvious way is to create a dense vector, eg.

In [6]:
x = randn(5) # vector we want to reflect
v = x/norm(x) # normalise x
Q = I - 2v*v' # a reflection matrix

5×5 Matrix{Float64}:
  0.210752     0.110274     0.00816022  -0.472562     0.848553
  0.110274     0.984592    -0.00114015   0.0660267   -0.11856
  0.00816022  -0.00114015   0.999916     0.00488593  -0.00877339
 -0.472562     0.0660267    0.00488593   0.717054     0.508071
  0.848553    -0.11856     -0.00877339   0.508071     0.0876856

Note `I` is a special convenience type that represents the identity matrix for any dimension.

A special type of reflection is a Householder reflection, which maps a vector to the $x$-axis.
Using dense matrices we can construct it as follows:

In [7]:
function dense_householderreflection(x)
    y = copy(x)
    if x[1] == 0
        y[1] += norm(x)
    else # note sign(z) = exp(im*angle(z)) where `angle` is the argument of a complex number
        y[1] += sign(x[1])*norm(x)
    end
    w = y/norm(y)
    I - 2*w*w'
end


x = randn(3) + im*randn(3)
Q = dense_householderreflection(x)
Q * x # all the entries apart from the first are numerically zero

3-element Vector{ComplexF64}:
     0.532414809876821 - 2.0399771891593383im
 2.220446049250313e-16 - 1.3877787807814457e-17im
                   0.0 + 2.220446049250313e-16im

A matrix-vector product is $O(n^2)$ operations but we know we can reduce it to $O(n)$.
Thus we will create a special type to represent the reflection and obtain the better complexity
multiplication. Because we want the matrix to be real when the entries are real we will use
a special feature called "templating". Here by adding the `{T}` after the type we allow this to
be either a `Float64` or `ComplexF64` (or indeed a `BigFloat`). We also do some checking
to make sure that our defining vector is already normalised.

In [8]:
struct Reflection{T} <: AbstractMatrix{T}
    v::Vector{T} # T can be either a Float64 or ComplexF64
end

function Reflection(v::Vector)
    T = eltype(v) # find the type of the entries of v
    if !(norm(v) ≈ 1)
        error("input must be normalised")
    end
    Reflection{T}(v) # create an instance of Reflection, specifying the entry type
end


## Implementations of Reflection * AbstractMatrix
# You may wish to use this below to solve Problem 3.
function *(Q::Reflection, X::AbstractMatrix)
    T = promote_type(eltype(Q), eltype(X))
    m,n = size(X)
    ret = zeros(T, m, n)
    for j = 1:n
        ret[:,j] = Q * X[:,j]
    end
    ret
end

* (generic function with 367 methods)

-----

**Problem 2(a)** Complete the implementation of a type representing an n × n
reflection that supports `Q[k,j]` in $O(1)$ operations and `*` in $O(n)$ operations.
The reflection may be complex (that is, $Q ∈ U(n)$ is unitary).

In [9]:
# Represents I - 2v*v'


size(Q::Reflection) = (length(Q.v),length(Q.v))

# getindex(Q, k, j) is synonym for Q[k,j]
function getindex(Q::Reflection, k::Int, j::Int)
    a = -2*conj(Q.v[j])*Q.v[k]
    if k == j
        a += 1
    end
    a
end
function *(Q::Reflection, x::AbstractVector)
    y = copy(x)
    y -= 2*Q.v*(Q.v'*x)
    y
end

# If your code is correct, these "unit tests" will succeed
n = 10
x = randn(n) + im*randn(n)
v = x/norm(x)
Q = Reflection(v)
@test Q == I-2v*v'
@test Q'Q ≈ I


# We can scale to very large sizes. here we check the reflection property on an 100_000 matrix:
n = 100_000
x = randn(n) + im*randn(n)
v = x/norm(x)
Q = Reflection(v)
@test Q*x ≈ -x

Test Passed
  Expression: Q * x ≈ -x
   Evaluated: ComplexF64[2.1538066082684026 - 0.18948119873430833im, 0.875841673399823 + 0.23112831700500755im, 0.8694330396234974 + 0.2799382407560835im, -1.5172295191471514 - 1.7019437668374024im, -0.15789222385191362 + 1.1994446476503842im, 1.9626569437261665 + 0.6302831668815558im, -2.203720976128073 + 0.586349317578734im, -1.9008268419515897 + 1.0451506507035588im, 0.4576784590988926 + 1.0903259869186546im, -0.2530416194070889 - 0.5182975900196231im  …  -0.7004168836887648 + 0.674327555302674im, 0.9216402882947269 + 0.2951775947347042im, 1.0374070602162253 + 1.0290319897308706im, 0.1259559089882645 - 2.0579485752274618im, -0.4930732812407749 + 0.6987763488712927im, 0.4991905430510914 + 1.06148125081347im, -0.19830267753857128 - 0.2962361225015208im, 0.08181010254334593 - 0.5062292176282687im, -1.118959481268426 - 1.119734955640869im, -0.13562617384412073 + 0.08522804003865939im] ≈ ComplexF64[2.1538066082684035 - 0.18948119873430838im, 0.8758416

**Problem 2(b)** Complete the following implementation of a Housholder reflection  so that the
unit tests pass, using the `Reflection` type created above.
Here `s == true` means the Householder reflection is sent to the positive axis and `s == false` is the negative axis.
You may assume `x` has real entries.

In [10]:
function householderreflection(s::Bool, x::AbstractVector)
    # TODO: return a Reflection corresponding to a Householder reflection
    y = copy(x) # don't modify x
    if s
        y[1] -= norm(x)
    else
        y[1] += norm(x)
    end
    Reflection(y/norm(y))
end

x = randn(5)
Q = householderreflection(true, x)
@test Q isa Reflection
@test Q*x ≈ [norm(x);zeros(eltype(x),length(x)-1)]

Q = householderreflection(false, x)
@test Q isa Reflection
@test Q*x ≈ [-norm(x);zeros(eltype(x),length(x)-1)]

Test Passed
  Expression: Q * x ≈ [-(norm(x)); zeros(eltype(x), length(x) - 1)]
   Evaluated: [-1.3562034803928689, 0.0, 0.0, 0.0, 0.0] ≈ [-1.3562034803928689, 0.0, 0.0, 0.0, 0.0]

**Problem 2(c)**
Complete the definition of `Reflections` which supports a sequence of reflections,
that is,
$$
Q = Q_{𝐯_1} ⋯ Q_{𝐯_m}
$$
where the vectors are stored as a matrix $V ∈ ℂ^{n × m}$ whose $j$-th column is $𝐯_j∈ ℂ^n$, and
$$
Q_{𝐯_j} = I - 2 𝐯_j 𝐯_j^⋆
$$
is a reflection.

In [11]:
struct Reflections{T} <: AbstractMatrix{T}
    V::Matrix{T} # Columns of V are the householder vectors
end

size(Q::Reflections) = (size(Q.V,1), size(Q.V,1))


function *(Q::Reflections, x::AbstractVector)
    # TODO: Apply Q in O(mn) operations by applying
    # the reflection corresponding to each column of Q.V to x
    m,n = size(Q.V)
    for j = n:-1:1
        x = Reflection(Q.V[:, j]) * x
    end
    x
end


## Implementations of Reflections * AbstractMatrix
# You may wish to use this below to solve Problem 3.
function *(Q::Reflections, X::AbstractMatrix)
    T = promote_type(eltype(Q), eltype(X))
    m,n = size(X)
    ret = zeros(T, m, n)
    for j = 1:n
        ret[:,j] = Q * X[:,j]
    end
    ret
end


function getindex(Q::Reflections, k::Int, j::Int)
    T = eltype(Q.V)
    m,n = size(Q)
    eⱼ = zeros(T, m)
    eⱼ[j] = one(T)
    return (Q*eⱼ)[k]
end

import LinearAlgebra: adjoint
function adjoint(Q::Reflections) # called when calling Q'
    Reflections(Q.V[:,end:-1:1])
end

Y = randn(5,3)
V = Y * Diagonal([1/norm(Y[:,j]) for j=1:3])
Q = Reflections(V)
@test Q ≈ (I - 2V[:,1]*V[:,1]')*(I - 2V[:,2]*V[:,2]')*(I - 2V[:,3]*V[:,3]')
@test Q' isa Reflections
@test Q' ≈ (I - 2V[:,3]*V[:,3]')*(I - 2V[:,2]*V[:,2]')*(I - 2V[:,1]*V[:,1]')
@test Q'Q ≈ I

Test Passed
  Expression: Q' * Q ≈ I
   Evaluated: [1.0000000000000002 4.163336342344337e-17 … 1.942890293094024e-16 -1.1102230246251565e-16; 1.3877787807814457e-16 0.9999999999999997 … 4.996003610813204e-16 -2.220446049250313e-16; … ; 5.551115123125783e-17 1.6653345369377348e-16 … 0.9999999999999993 0.0; 5.551115123125783e-17 -1.6653345369377348e-16 … -1.1102230246251565e-16 0.9999999999999998] ≈ UniformScaling{Bool}(true)

-----

## III.6 QR Factorisation

The QR factorisation of a matrix $A ∈ ℂ^{m × n}$ is of the form
$$
A = QR
$$
where $Q$ is unitary and $R$ is right-triangular: all entries below the diagonal are zero. We focus on the case where $m ≥ n$.
It can be computed using Gram–Schmidt, Householder reflections or rotations.

### III.6.1 Reduced QR and Gram–Schmidt

The Gram–Schmidt process can be used to compute the QR factorisation by orthogonalising the columns
of $A$ in sequence. We won't discuss this in more detail as it is numerically better to use reflections/rotations.

### III.6.2 Householder reflections and QR

In the notes we use Householder reflections to prove that a QR factorisation exists.
The iterative proof actually encodes an algorithm, which we can implement as follows:

In [12]:
function householderqr(A)
    T = eltype(A)
    m,n = size(A)
    if n > m
        error("More columns than rows is not supported")
    end

    R = zeros(T, m, n)
    Q = Matrix(one(T)*I, m, m)
    Aⱼ = copy(A)

    for j = 1:n
        𝐚₁ = Aⱼ[:,1] # first columns of Aⱼ
        Q₁ = dense_householderreflection(𝐚₁)
        Q₁Aⱼ = Q₁*Aⱼ # multiply Aⱼ by the Householder reflection
        α,𝐰 = Q₁Aⱼ[1,1],Q₁Aⱼ[1,2:end]

        # populate returned data
        R[j,j] = α
        R[j,j+1:end] = 𝐰

        # following is equivalent to Q = Q*[I 0 ; 0 Qⱼ]
        Q[:,j:end] = Q[:,j:end]*Q₁
        
        Aⱼ = Q₁Aⱼ[2:end,2:end] # this is the "induction"
    end
    Q,R
end

m,n = 100,50
A = randn(m,n)
Q,R = householderqr(A)
@test Q'Q ≈ I
@test Q*R ≈ A

Test Passed
  Expression: Q * R ≈ A
   Evaluated: [-0.722122766210186 -0.7531853830931851 … -0.10958267528674034 -0.28604026090033485; 0.05551734734398218 1.698944381518207 … -1.0136353488760028 1.119739920090353; … ; -0.23862846875933177 0.16989564706393862 … 0.46031779574201775 -0.25706198581613393; -0.1684909214567553 0.47053387639692923 … 0.7871512373372883 0.21494921602741235] ≈ [-0.7221227662101831 -0.7531853830931861 … -0.10958267528674327 -0.28604026090033463; 0.05551734734398216 1.6989443815182066 … -1.013635348876002 1.1197399200903455; … ; -0.23862846875933166 0.16989564706393867 … 0.4603177957420168 -0.2570619858161331; -0.16849092145675523 0.47053387639692945 … 0.7871512373372874 0.21494921602741154]

Note because we are forming a full matrix representation of each Householder
reflection this is a slow algorithm: it uses $O(m^2 n^2)$ operations, which is too many!
By being more careful about how we apply and store reflections we can avoid this,
in particular, taking advantage of the types `Reflection` and `Reflections`.

------

**Problem 3** Complete the following function that implements
Householder QR for a real matrix $A ∈ ℝ^{m × n}$ where $m ≥ n$ using only $O(mn^2)$ operations, using
 `Reflection` and `Reflections`.

In [25]:

function householderqr(A)
    T = eltype(A)
    m,n = size(A)
    if n > m
        error("More columns than rows is not supported")
    end

    R = zeros(T, m, n)
    Q = Reflections(zeros(T, m, n))
    Aⱼ = copy(A)

    for j = 1:n
        # TODO: rewrite householder QR to use Reflection,
        # Reflections and householderreflection, in a way that one achieves O(mn^2) operations
        # SOLUTION
        𝐚₁ = Aⱼ[:,1] # first columns of Aⱼ
        Q₁ = householderreflection(𝐚₁[1] < 0, 𝐚₁)
        Q₁Aⱼ = Q₁*Aⱼ
        α,𝐰 = Q₁Aⱼ[1,1],Q₁Aⱼ[1,2:end]
        Aⱼ₊₁ = Q₁Aⱼ[2:end,2:end]

        # populate returned data
        R[j,j] = α
        R[j,j+1:end] = 𝐰

        Q.V[j:end, j] = Q₁.v

        Aⱼ = Aⱼ₊₁ # this is the "induction"
        # END
    end
    Q,R
end

A = randn(600,400)
Q,R = householderqr(A)
@test Q*R ≈ A

Test Passed
  Expression: Q * R ≈ A
   Evaluated: [0.6012825715531456 0.8069294723789866 … -1.534268755678 -0.2976238425546468; -0.10433162179817825 -1.2364897555243346 … 0.26914247563753513 -0.022082130469634493; … ; 0.25531973324548984 1.1082408718064165 … 1.4534176274810668 -1.2592141851539476; 0.10424008498863459 -1.4544143803698855 … -0.6909979584772324 1.0888751307088929] ≈ [0.6012825715531508 0.8069294723789875 … -1.5342687556780004 -0.2976238425546462; -0.10433162179817827 -1.2364897555243388 … 0.26914247563753546 -0.02208213046963413; … ; 0.2553197332454899 1.1082408718064167 … 1.4534176274810648 -1.259214185153949; 0.1042400849886346 -1.4544143803698857 … -0.6909979584772314 1.0888751307088942]

------
### Given's Rotations and QR

An alternative to using reflections to introduce zeros is to use rotations, which
are called Given's Rotations.
This is particularly convenient for tridiagonal matrices, where one needs to only
make one sub-diagonal zero. Here we explore a tridiagonal QR built from rotations
in a way that the factorisation can be computed in $O(n)$ operations.

-----

**Problem 4** This problem explores computing  a QR factorisation of a Tridiagonal matrix in $O(n)$ operations.
This will introduce entries in the second super-diagonal, hence we will use the `UpperTridiagonal` type
from Lab 6 (solution copied below). Complete the implementation of `bandedqr`, that only takes $O(n)$ operations,
using an instance of `Reflections` to represent `Q` and `UpperTriangular` to represent `R`.

In [19]:
import Base: *, size, getindex, setindex!
struct UpperTridiagonal{T} <: AbstractMatrix{T}
    d::Vector{T}   # diagonal entries
    du::Vector{T}  # super-diagonal enries
    du2::Vector{T} # second-super-diagonal entries
end

size(U::UpperTridiagonal) = (length(U.d),length(U.d))

function getindex(U::UpperTridiagonal, k::Int, j::Int)
    d,du,du2 = U.d,U.du,U.du2
    if j - k == 0
        d[j]
    elseif j - k == 1
        du[k]
    elseif j - k == 2
        du2[k]
    else
        0
    end
end

function setindex!(U::UpperTridiagonal, v, k::Int, j::Int)
    d,du,du2 = U.d,U.du,U.du2
    if j > k+2
        error("Cannot modify off-band")
    end
    if j - k == 0
        d[k] = v
    elseif j - k == 1
        du[k] = v
    elseif j - k == 2
        du2[k] = v
    else
        error("Cannot modify off-band")
    end
    U # by convention we return the matrix
end


function bandedqr(A::Tridiagonal)
    n = size(A, 1)
    Q = Rotations(zeros(n - 1)) # Assume Float64
    R = UpperTridiagonal(zeros(n), zeros(n - 1), zeros(n - 2))

    # TODO: Populate Q and R by looping through the columns of A.
    R[1, 1:2] = A[1, 1:2]

    for j = 1:n-1
        x_1 = [R[j, j], A[j+1, j]]
        Q.θ[j] = atan(x_1[2], x_1[1])
        Q_1 = Rotation(-Q.θ[j])
        R[j, j] = (Q_1 * x_1)[1]

        x_2 = [R[j, j+1]; A[j+1, j+1]]
        R[j:j+1, j+1] = Q_1 * x_2

        if j < n - 1
            x_3 = [0, A[j+1, j+2]]
            R[j:j+1, j+2] = Q_1 * x_3
        end
    end

    Q, R
end

A = Tridiagonal([1, 2, 3, 4], [1, 2, 3, 4, 5], [1, 2, 3, 4])
Q, R = bandedqr(A)
@test Q*R ≈ A

Test Passed
  Expression: Q * R ≈ A
   Evaluated: [1.0 1.0000000000000002 … 4.440892098500626e-16 2.220446049250313e-16; 0.9999999999999998 1.9999999999999996 … -4.440892098500626e-16 0.0; … ; 0.0 0.0 … 4.0 3.9999999999999996; 0.0 0.0 … 4.0 5.0] ≈ [1 1 … 0 0; 1 2 … 0 0; … ; 0 0 … 4 4; 0 0 … 4 5]

### III.6.3 QR and least squares

When we type `A \ b` with a rectangular matrix `A` it is
solving a least squares system, and behind the scenes it is using a QR factorisation.
We can see this via the inbulit `qr` function

In [36]:
A = randn(200,200)
b = randn(200)

Q,R̂ = qr(A)

LinearAlgebra.QRCompactWY{Float64, Matrix{Float64}}
Q factor:
200×200 LinearAlgebra.QRCompactWYQ{Float64, Matrix{Float64}}:
 -0.090397     0.0349234    0.0522127   …  -0.0784188   -0.113031
  0.128346    -0.116031    -0.0371368      -0.0240599   -0.0753319
  0.0185748   -0.00456704  -0.0290237       0.0131451    0.00214515
 -0.033694     0.0940443   -0.0861065      -0.058698     0.0147346
 -0.0222219   -0.0136734   -0.077512       -0.0110547   -0.0187578
  0.0527347   -0.0231176   -0.0423819   …   0.0200723    0.0205715
 -0.0243438    0.00467213   0.0724933      -0.0738947    0.00411624
 -0.0096651    0.117121     0.0268756       0.0641198   -0.0544105
  0.0155246    0.134026     0.00887688      0.0555357   -0.0520731
 -0.0500381   -0.185573    -0.171882        0.0657933   -0.0121943
  ⋮                                     ⋱               
 -0.00191908   0.0722474   -0.103801        0.122259    -0.0840985
 -0.0129383   -0.0453873    0.0143354       0.0859418   -0.0300949
  0.0168615   

Here `Q` is a special type representing an orthogonal matrix.
`R̂` is an `UpperTriangular`, that is, we only store the upper triangular
entries of `R` (which is the same as the reduced QR factorisation).
Thus to solve a least squares problem we need to drop the extra entries as
follows:

In [33]:
c = Q'b # invert Q
c̃ = c[1:size(R̂,1)] # drop extra entries
@test A \ b ≈ R̂ \ c̃

DimensionMismatch: DimensionMismatch("dot product arguments have lengths 600 and 200")

**Problem 5** Complete the function `leastsquares(A, b)` that uses your
`householderqr` function to solve a least squares problem.

In [37]:
function leastsquares(A, b)
    # TODO: use householderqr to solve a least squares problem.
    m,n = size(A)
    Q, R = householderqr(A)
    UpperTriangular(R[1:n,1:n])\(Q'b)[1:n]
end

@test A\b ≈ leastsquares(A,b)

Test Passed
  Expression: A \ b ≈ leastsquares(A, b)
   Evaluated: [0.2076089874803123, -1.5781490783413468, 0.3212273248647892, 0.8297465567801846, -1.0226846749779552, 2.311814224025939, 1.571050464849211, -2.0117093369421526, 0.19847639908373518, 0.10046229746289151  …  2.1284395913820644, 1.0833593770451524, 0.1930947613487933, -1.747447962305926, -1.6468976394537547, -1.6150949857567785, 2.8028543990135626, -0.8731367906119285, -0.41005708518225215, 3.964660537694465] ≈ [0.2076089874803031, -1.578149078341378, 0.3212273248648203, 0.8297465567802076, -1.022684674978001, 2.311814224025943, 1.5710504648492318, -2.011709336942161, 0.1984763990837821, 0.10046229746291392  …  2.1284395913821155, 1.0833593770451526, 0.19309476134878123, -1.7474479623059949, -1.6468976394537918, -1.6150949857568013, 2.8028543990136083, -0.873136790611929, -0.41005708518227946, 3.964660537694527]

---

*This notebook was generated using [Literate.jl](https://github.com/fredrikekre/Literate.jl).*